# MoodStream: Эксперименты 

Основная задача заключается в подборе модели с удовлетворительным процентом ошибок и приемлемым временем выполнения.

---

## Импорты

In [1]:
import pandas as pd
import numpy as np
import requests, zipfile, io
from sklearn.metrics.pairwise import cosine_similarity
import dotenv

from clearml import Dataset

## ENV

In [2]:
# В корне необходим файл .env, устанавливающий значения для переменных CLEARML_API_ACCESS_KEY и CLEARML_API_SECRET_KEY
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
    
%load_ext dotenv
%dotenv

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml


## Данные

### Фильмы

In [3]:
movies_dataset = Dataset.get(
    dataset_id='9059bbbf193b4aa688ab63642e489ea8',  
    only_completed=True, 
    auto_create=False
)
movies_dataset_local_path = movies_dataset.get_local_copy()
movies_data_df = pd.read_csv(f'{movies_dataset_local_path}/movies.csv', sep='\t')
movies_ratings_df = pd.read_csv(f'{movies_dataset_local_path}/ratings.csv', sep='\t')
movies_genres_df = pd.read_csv(f'{movies_dataset_local_path}/genres.csv', sep='\t')
print('MOVIES:')
print(movies_data_df.head(5))
print('RATINGS:')
print(movies_ratings_df.head(5))
print('GENRES:')
print(movies_genres_df.head(5))

MOVIES:
                                              poster  \
0  https://m.media-amazon.com/images/M/MV5BMDFkYT...   
1  https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2  https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3  https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4  https://m.media-amazon.com/images/M/MV5BMWU4N2...   

                      title  year  imdb_rating  movie_id  \
0  The Shawshank Redemption  1994          9.3       318   
1             The Godfather  1972          9.2       858   
2           The Dark Knight  2008          9.0     58559   
3    The Godfather: Part II  1974          9.0      1221   
4              12 Angry Men  1957          9.0      1203   

                        genres  
0                  drama,crime  
1                  drama,crime  
2  drama,action,crime,thriller  
3                  drama,crime  
4                        drama  
RATINGS:
   user_id  movie_id  rating   like
0    58365        32     5.0  False
1    58365     

### Книги

In [4]:
# books_dataset = Dataset.get(
#     dataset_id='3f5f71caa9ae4709b827c5dd88e7253a',  
#     only_completed=True, 
#     auto_create=False
# )
# books_dataset_local_path = books_dataset.get_local_copy()
# books_data_df = pd.read_csv(f'{books_dataset_local_path}/books.csv', sep='\t')
# books_ratings_df = pd.read_csv(f'{books_dataset_local_path}/ratings.csv', sep='\t')
# books_genres_df = pd.read_csv(f'{books_dataset_local_path}/genres.csv', sep='\t')
# print('BOOKS:')
# print(books_data_df.head(5))
# print('RATINGS:')
# print(books_ratings_df.head(5))
# print('GENRES:')
# print(books_genres_df.head(5))

### Музыка

In [5]:
# tracks_dataset = Dataset.get(
#     dataset_id='dc72aebbbee64810a590a4c5bd82c19b',  
#     only_completed=True, 
#     auto_create=False
# )
# tracks_dataset_local_path = tracks_dataset.get_local_copy()
# tracks_data_df = pd.read_csv(f'{tracks_dataset_local_path}/tracks.csv', sep='\t')
# tracks_ratings_df = pd.read_csv(f'{tracks_dataset_local_path}/ratings.csv', sep='\t')
# tracks_genres_df = pd.read_csv(f'{tracks_dataset_local_path}/genres.csv', sep='\t')
# print('TRACKS:')
# print(tracks_data_df.head(5))
# print('RATINGS:')
# print(tracks_ratings_df.head(5))
# print('GENRES:')
# print(tracks_genres_df.head(5))

## User-based коллаборативная фильтрация

In [ ]:
# создание матрицы пользователь-фильм
user_movie_matrix = movies_ratings_df.pivot_table(index='user_id', columns='movie_id', values='rating')

# заполнение пропущенных значений нулями
user_movie_matrix = user_movie_matrix.fillna(0)

# расчет сходства пользователей
user_similarity = cosine_similarity(user_movie_matrix)

# функция для предсказания оценки пользователя для фильма
def predict_rating(user_movie_matrix, user_similarity, user_id, movie_id, k):
    # получение оценок пользователя
    user_ratings = user_movie_matrix.loc[user_id]
    
    # расчет сходства пользователя с другими пользователями
    similarity_scores = pd.Series(user_similarity[user_id])
    
    # фильтрация пользователей, которые оценили данный фильм
    similar_users = user_movie_matrix[movie_id][user_movie_matrix[movie_id] > 0].index
    
    # фильтрация k наиболее похожих пользователей
    similar_users = similarity_scores[similarity_scores > 0].sort_values(ascending=False).head(k).index
    
    # получение оценок похожих пользователей для данного фильма
    similar_ratings = user_movie_matrix.loc[similar_users, movie_id]
    
    # расчет взвешенного среднего рейтинга на основе оценок похожих пользователей
    if len(similar_ratings) > 0:
        weighted_ratings = similar_ratings * similarity_scores.loc[similar_users]
        predicted_rating = weighted_ratings.sum() / similarity_scores.loc[similar_users].sum()
    else:
        predicted_rating = user_ratings.mean()
    
    return predicted_rating

# пример использования функции для предсказания оценки пользователя для фильма
user_id = 58365
movie_id = 32
k = 10
predicted_rating = predict_rating(user_movie_matrix, user_similarity, user_id, movie_id, k)
print('Predicted rating for user', user_id, 'and movie', movie_id, ':', predicted_rating)
